In [3]:
!mkdir densephrase_out
!mkdir /home/miu/Projects/NLP/densephrase_out/cache

mkdir: cannot create directory ‘/home/miu/Projects/NLP/densephrase_out/cache’: File exists


In [6]:
#pip install faiss-gpu blosc spacy ujson
!python generate_phrase_vecs.py \
		--pretrained_name_or_path SpanBERT/spanbert-base-cased \
		--cache_dir /home/miu/Projects/NLP/densephrase_out/cache \
		--do_dump \
		--max_seq_length 512 \
		--doc_stride 462 \
		--fp16 \
		--filter_threshold -2.0 \
		--append_title \
		--test_file /home/miu/Projects/NLP/densephrases-data/single-qa/squad/test.json \
		--load_dir /home/miu/Projects/NLP/densephrases-multi/ \
		--output_dir /home/miu/Projects/NLP/densephrase_out/test_model 

01/06/2024 22:15:49 - WARNING - densephrases.options -   WARNING - You've set a doc stride which may be superior to the document length in some examples.
01/06/2024 22:15:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/06/2024 22:15:49 - INFO - __main__ -   Dump parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_mod

In [7]:
mkdir -p /home/miu/Projects/NLP/densephrase_out/logs

In [11]:
!mkdir /home/miu/Projects/NLP/densephrase_out/index
!mkdir /home/miu/Projects/NLP/densephrase_out/index/phrase

mkdir: cannot create directory ‘/home/miu/Projects/NLP/densephrase_out/index’: File exists


In [10]:
!python build_phrase_index.py \
		--dump_dir /home/miu/Projects/NLP/densephrase_out/test_model/dump \
		--index_name myindex \
		--stage all \
		--replace \
		--num_clusters 256 \
		--fine_quant OPQ96 \
		--doc_sample_ratio 1 \
		--cuda
#doc_sample_ratio so para teste, pois so tem 1 documento no teste

01/07/2024 11:15:51 - INFO - __main__ -   Creating 256_flat_OPQ96...
sampling from:
/home/miu/Projects/NLP/densephrase_out/test_model/dump/phrase/test.hdf5
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 15.24it/s]
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 7794 points to 256 centroids: please provide at least 9984 training points
WARNING clu

In [2]:
#compress metada
!python compress_metadata.py \
		--input_dump_dir /home/miu/Projects/NLP/densephrase_out/test_model/dump/phrase \
		--output_dir /home/miu/Projects/NLP/densephrase_out/test_model/dump

loading doc groups: 100%|████████████████████████| 1/1 [00:00<00:00, 208.36it/s]
compressed by 22.67%


In [5]:
# 5) Evaluate the phrase index for phrase retrieval
!python eval_phrase_retrieval.py \
		--run_mode eval \
		--pretrained_name_or_path SpanBERT/spanbert-base-cased \
		--cuda \
		--cache_dir /home/miu/Projects/NLP/densephrase_out/cache \
		--dump_dir /home/miu/Projects/NLP/densephrase_out/test_model/dump \
		--index_name myindex \
		--load_dir /home/miu/Projects/NLP/densephrases-multi/ \
		--test_path /home/miu/Projects/NLP/densephrases-data/single-qa/squad/test.json \
		--save_pred \
		--aggregate 

Traceback (most recent call last):
  File "eval_phrase_retrieval.py", line 391, in <module>
    evaluate(args)
  File "eval_phrase_retrieval.py", line 51, in evaluate
    qids, questions, answers, _ = load_qa_pairs(args.test_path, args, q_idx)
  File "/home/miu/Projects/NLPLab/DensePhrasesMinimal/densephrases/utils/open_utils.py", line 108, in load_qa_pairs
    q_id = item['id']
KeyError: 'id'


In [1]:
import h5py

f=h5py.File('/home/miu/Projects/NLP/densephrase_out/test_model/dump/phrase/test.hdf5', 'r')

In [9]:
import random
doc_ids = list(f.keys())
doc_sample_ratio=1
sampled_doc_ids = random.sample(doc_ids, int(doc_sample_ratio * len(doc_ids)))
sampled_doc_ids

['0']